In [1]:
import cupy as cp

def random_vectors_on_unit_sphere(num_samples):
    phi = cp.random.uniform(0, 2 * cp.pi, num_samples)
    costheta = cp.random.uniform(-1, 1, num_samples)
    
    theta = cp.arccos(costheta)
    x = cp.sin(theta) * cp.cos(phi)
    y = cp.sin(theta) * cp.sin(phi)
    z = costheta

    return cp.vstack([x, y, z]).T

def is_within_cone_batch(vectors, origins, angle):
    """
    Check if a set of vectors are within cones defined by angles from origin vectors.
    Returns a mask of shape (len(vectors), len(origins)).
    """
    cos_angle = cp.cos(cp.radians(angle))
    # Use broadcasting to compute the dot product between vectors and each origin
    dot_products = cp.einsum('ik,jk->ij', vectors, origins)
    return dot_products > cos_angle

def generate_vectors_in_cone(origins, angle, num_samples):
    num_origins = origins.shape[0]
    estimate_samples = num_samples * 5
    
    vectors = random_vectors_on_unit_sphere(estimate_samples)
    mask = is_within_cone_batch(vectors, origins, angle)
    
    samples_in_cones = []
    for origin_idx in range(num_origins):
        origin_mask = mask[:, origin_idx]
        samples_for_origin = vectors[origin_mask][:num_samples]
        
        # If not enough samples, recursively call to gather more
        while samples_for_origin.shape[0] < num_samples:
            additional_samples = generate_vectors_in_cone(cp.array([origins[origin_idx]]), angle, num_samples - samples_for_origin.shape[0])
            samples_for_origin = cp.vstack([samples_for_origin, additional_samples[:num_samples - samples_for_origin.shape[0]]])
        
        samples_in_cones.append(samples_for_origin)

    return samples_in_cones

# Usage:
origins = cp.array([[0, 0, 1], [0, 1, 0]])  # 2 origin vectors as an example
samples = generate_vectors_in_cone(origins, 30, 1000)  # Generate 1000 samples for each origin vector


ModuleNotFoundError: No module named 'cupy'

In [2]:
import numpy as np

def random_vectors_on_unit_sphere(num_samples):
    phi = np.random.uniform(0, 2 * np.pi, num_samples)
    costheta = np.random.uniform(-1, 1, num_samples)
    
    theta = np.arccos(costheta)
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = costheta

    return np.vstack([x, y, z]).T

def is_within_cone_batch(vectors, origins, angle):
    """
    Check if a set of vectors are within cones defined by angles from origin vectors.
    Returns a mask of shape (len(vectors), len(origins)).
    """
    cos_angle = np.cos(np.radians(angle))
    # Use broadcasting to compute the dot product between vectors and each origin
    dot_products = np.einsum('ik,jk->ij', vectors, origins)
    return dot_products > cos_angle

def generate_vectors_in_cone(origins, angle, num_samples):
    num_origins = origins.shape[0]
    estimate_samples = num_samples * 5
    
    vectors = random_vectors_on_unit_sphere(estimate_samples)
    mask = is_within_cone_batch(vectors, origins, angle)
    
    samples_in_cones = []
    for origin_idx in range(num_origins):
        origin_mask = mask[:, origin_idx]
        samples_for_origin = vectors[origin_mask][:num_samples]
        
        # If not enough samples, recursively call to gather more
        while samples_for_origin.shape[0] < num_samples:
            additional_samples = generate_vectors_in_cone(np.array([origins[origin_idx]]), angle, num_samples - samples_for_origin.shape[0])
            samples_for_origin = np.vstack([samples_for_origin, additional_samples[0][:num_samples - samples_for_origin.shape[0]]])
        
        samples_in_cones.append(samples_for_origin)

    return samples_in_cones

# Usage:
origins = np.array([[0, 0, 1], [0, 1, 0]])  # 2 origin vectors as an example
samples = generate_vectors_in_cone(origins, 30, 1000)  # Generate 1000 samples for each origin vector


In [7]:
print(samples[0])
print(samples[1])

[[-0.43685201 -0.12363513  0.89099645]
 [-0.14018235 -0.38109922  0.91384479]
 [ 0.03726318 -0.0956744   0.99471497]
 ...
 [ 0.34056799 -0.15628012  0.92714075]
 [-0.25490187  0.395909    0.88220241]
 [-0.29603741 -0.19809584  0.93440884]]
[[-0.08304653  0.88481279 -0.45848621]
 [ 0.35281244  0.88300504  0.30955691]
 [ 0.19372016  0.93477213 -0.29778105]
 ...
 [-0.11795208  0.94422339  0.30745648]
 [ 0.17244257  0.93424443 -0.31217128]
 [-0.00921484  0.99239606 -0.12274019]]
